In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re, string, sys, pickle, json

In [2]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
remove_after = r' as in | refers to | etc.| also written | also known | e\.g\.|; |–|—| that is | that is,'
split_by = r'<br\/>| \/ |\n| \\ | or '
acr_punc  = '!"\',.:;<=>?`#%()[]{}'
note_punc = '!"\',.:;<=>?`#%()[]{}'
peroid = r'\.\s+'

In [5]:
def add_to_dict(url, dct):
    global k
    website_url = requests.get(url).text

    soup = BeautifulSoup(website_url,'lxml')
#     print(soup.prettify())
    
    My_table = soup.find('table',{'class':'wikitable sortable'})
    
    parsed_table = soup.find_all('table')[1] 
    data = [[td for td in row.find_all('td')]
            for row in parsed_table.find_all('tr')]
    
    def clean_acr(html_text, is_meaning=None):
        removed_braces =    re.sub(r'\b\d+\b', ' number_phrase ',    # replace numbers
                            re.sub(r'\{(?:[^}{]|\{[^}{]*\})*\}', '', # remove nested {}
                            re.sub(r'\[(?:[^][]|\[[^][]*\])*\]', '', # remove nested []
                            re.sub(r'\((?:[^)(]|\([^)(]*\))*\)', '', # remove nested ()
                            str(html_text)))))
        
        acrs = set()
        # get each acr
        for split_html in re.split(split_by, removed_braces):
            ss = BeautifulSoup(split_html, "lxml")

            # clean data
            acr = re.split(remove_after, # remove everything after something in remove_after
                    ss.get_text().lower() # extract text and lower it
                    )[0].translate({ord(c): None for c in acr_punc} # remove punc
                    ).strip(  # lower and strip leading/trailing whitespace
                    ).encode('ascii','ignore')  # remove unicode characters

            
                
#             print(acr, is_meaning, '******', len(acr.split(' ')))
            if (len(acr)>0 and
                ((is_meaning and len(acr.split(' ')) <= 5) or len(acr.split(' ')) <= 1)):
#                 print(acr, is_meaning)
                acrs.add(acr)
#             if len(acr.split(' ')) > 5:
#                 print(is_meaning, acr, ss.get_text().lower())
            
        return list(acrs)

                    
                    
                    
#         print (ss.get_text().lower())
#         return re.split(remove_after, # remove everything after something in remove_after
#                     re.sub(r'\[(?:[^][]|\[[^][]*\])*\]', '', # remove nested []
#                     re.sub(r'\((?:[^)(]|\([^)(]*\))*\)', '', # remove nested ()
#                     ss.get_text().lower()) # extract text and lower it
#                     ).translate(str.maketrans('', '', acr_punc) # remove punc
#                     ))[0].strip() # lower and strip leading/trailing whitespace
        

    for j in range(len(data)):
        if len(data[j])>0:
            acrs = clean_acr(data[j][0])

            # do the same for meanings
            meanings=clean_acr(data[j][1], is_meaning=acrs)
            if len(meanings) > 0:
                for acr in acrs:
                    if acr in dct:
                        dct[acr]+=meanings
#                         print (acr, meanings)
                    else:
                        dct[acr] = meanings
    
    
    return dct

In [6]:
dct = {}
for a in alphabet:
    url = 'https://en.wikipedia.org/wiki/List_of_medical_abbreviations:_'+a
    dct = add_to_dict(url, dct)
    print(a)

A
B
C
D
E
F
G
H
I
J
K
L
M
N
O
P
Q
R
S
T
U
V
W
X
Y
Z


In [7]:
dct['aa'] = ['amino acids']

In [8]:
with open('dictionary.txt') as f:
    eng_dict=set(f.read().split())

In [9]:
for acr in dct:
    if acr in eng_dict:
        dct[acr].append(acr)
        
for acr in dct:
    dct[acr] = list(set(dct[acr]))

In [10]:
'help' in dct, dct['aa'], dct['he'], dct['ed']

(False,
 ['amino acids'],
 ['he', 'hepatic encephalopathy'],
 ['effective dose',
  'emergency department',
  'ectodermal dysplasia',
  'erectile dysfunction',
  'emotional distress'])

In [11]:
with open('acros.pickle', 'wb') as handle:
    pickle.dump(dct, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# REST OF THIS IS JUST FOR PLAYING AROUND

In [ ]:
# REST OF THIS IS JUST FOR PLAYING AROUND

In [ ]:
# REST OF THIS IS JUST FOR PLAYING AROUND

In [10]:
flat_list = [item for sublist in dct.values() for item in sublist]
pd.DataFrame(flat_list).to_csv('test2.csv',encoding='utf-8-sig')

In [19]:
flat_list = [i for sublist in dct.values() for item in sublist for i in item.split()]
len(set(flat_list)), len(flat_list)

(2958, 7053)

In [20]:
flat_list = [item for sublist in dct.values() for item in sublist]
len(set(flat_list)), len(flat_list)

(2610, 2727)

In [11]:
dct

{u'frda': [u'friedreichs ataxia'],
 u'bc': [u'blood culture', u'bone conduction'],
 u'tibc': [u'total iron-binding capacity'],
 u'crtx': [u'chemoradiotherapy'],
 u'nwb': [u'non-weight-bearing'],
 u'auc': [u'area under the curve'],
 u'n&v': [u'nausea and vomiting'],
 u'spet': [u'single-photon emission tomography'],
 u'pth': [u'parathyroid hormone'],
 u'spep': [u'serum protein electrophoresis'],
 u'tme': [u'total mesorectal excision'],
 u'aur': [u'ear'],
 u'shbg': [u'sex hormone-binding globulin'],
 u'scd': [u'sickle-cell disease',
  u'sudden cardiac death',
  u'sequential compression device'],
 u'spec': [u'specimen'],
 u'orsa': [u'oxacillin-resistant staphylococcus aureus'],
 u'pacu': [u'post-anesthesia care unit'],
 u'kshv': [u'kaposis sarcoma-associated herpesvirus'],
 u'sca': [u'spinocerebellar ataxia'],
 u'ptb': [u'pulmonary tuberculosis'],
 u'scc': [u'squamous cell carcinoma'],
 u'cmo': [u'comfort measures only'],
 u'upo': [u'until proven otherwise'],
 u'upj': [u'ureteropelvic junc

In [14]:
with open('test.pickle', 'wb') as handle:
    json.dump(dct, handle)

In [ ]:
len(dct.keys())

In [168]:
tot=0
for i in dct.keys():
    tot+=len(i.split(' '))
    
for i in dct.values():
    for j in i:
        tot+=len(j.split(' '))
tot

10068

In [46]:
k

'(Note: Looks similar to, but does not mean, the chromium chlorides—CrCl2,'

In [69]:
re.sub(r'\((?:[^)(]|\([^)(]*\))*\)', '', '(Note: Looks similar to, but does not mean, the chromium chlorides—CrCl2,')

'(Note: Looks similar to, but does not mean, the chromium chlorides—CrCl2,'

In [75]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_medical_abbreviations:_A').text

In [76]:

soup = BeautifulSoup(website_url,'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of medical abbreviations: A - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_medical_abbreviations:_A","wgTitle":"List of medical abbreviations: A","wgCurRevisionId":888863599,"wgRevisionId":888863599,"wgArticleId":16981730,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Lists of medical abbreviations"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","

In [77]:
sslash='<td><a href="/wiki/Morphine" title="Morphine">morphine</a> sulfate (<b>Do not use</b> this abbreviation. Write out the name. Per the <a class="mw-redirect" href="/wiki/List_of_medical_abbreviations:_Do-not-use_list" title="List of medical abbreviations: Do-not-use list">do-not-use list</a>.)</td>'

ss = BeautifulSoup(sslash, "lxml")
acr = re.sub(r'[\(].*?[\)]', '', ss.get_text()).translate(str.maketrans('', '', acr_punc)).lower().strip()
acr, re.sub(r'[\(].*?[\)]', '', ss.get_text()), ss.get_text()


('morphine sulfate',
 'morphine sulfate ',
 'morphine sulfate (Do not use this abbreviation. Write out the name. Per the do-not-use list.)')

In [78]:
My_table = soup.find('table',{'class':'wikitable sortable'})
My_table

<table class="wikitable sortable">
<caption>Sortable table
</caption>
<tbody><tr>
<th scope="col">Abbreviation
</th>
<th scope="col">Meaning
</th></tr>
<tr>
<td>ā</td>
<td>(a with a bar over it) before (from <a href="/wiki/Latin" title="Latin">Latin</a> <i>ante</i>) <br/> <a class="extiw" href="https://en.wiktionary.org/wiki/before" title="wikt:before">before</a>
</td></tr>
<tr>
<td>A</td>
<td>assessment
</td></tr>
<tr>
<td>ap</td>
<td>apical
</td></tr>
<tr>
<td valign="top">a.a.</td>
<td>of each (from <a href="/wiki/Latin" title="Latin">Latin</a> <i>ana ana</i>) <br/> <a class="mw-redirect" href="/wiki/Amino_acids" title="Amino acids">amino acids</a>:
<ul><li>A or Ala – <a href="/wiki/Alanine" title="Alanine">alanine</a></li>
<li>C or Cys – <a href="/wiki/Cysteine" title="Cysteine">cysteine</a></li>
<li>D or Asp – <a href="/wiki/Aspartic_acid" title="Aspartic acid">aspartic acid</a></li>
<li>E or Glu – <a href="/wiki/Glutamic_acid" title="Glutamic acid">glutamic acid</a></li>
<li>F  o

In [79]:
t=pd.read_html(str(My_table))
t[0]

,0,1
0,Abbreviation,Meaning
1,ā,(a with a bar over it) before (from Latin ante...
2,A,assessment
3,ap,apical
4,a.a.,of each (from Latin ana ana) amino acids: A or...
5,A1C,glycosylated hemoglobin (hemoglobin A1c)
6,Aa.,"arteria, see Artery"
7,AAA,"abdominal aortic aneurysm (pronounced ""triple-A"")"
8,AAD,antibiotic-associated diarrhea
9,AAI,acute arterial insufficiency


In [80]:
parsed_table = soup.find_all('table')[1] 
data = [[td
         for td in row.find_all('td')]
        for row in parsed_table.find_all('tr')]

In [81]:
acr_punc = '!"\'(),.:;<=>?`'
note_punc = '!"\'(),:;<=>?`'

dct = {}
for j in range(len(data)):
    if len(data[j])>0:
        acrs = set()
         # get each acr
        for split_html in re.split('<br\/>| \/ |\n| \\ ', str(data[j][0])):
            ss = BeautifulSoup(split_html, "lxml")
             # get strip (stuff), then remove punc, then lower, then strip whitespace of ends
            acr = re.sub(r'\((?:[^)(]+|\((?:[^)(]+|\([^)(]*\))*\))*\)', '', ss.get_text()).translate(str.maketrans('', '', acr_punc)).lower().strip()
            print(acr)
            if len(acr)>0:
                acrs.add(acr)
        
        # do the same for meanings
        meanings=set()
        for split_html in re.split('<br\/>| \/ |\n| \\ ', str(data[j][1])):
            ss = BeautifulSoup(split_html, "lxml")
            meaning = re.sub(r'\((?:[^)(]+|\((?:[^)(]+|\([^)(]*\))*\))*\)', '', ss.get_text()).translate(str.maketrans('', '', acr_punc)).lower().strip()
            print(meaning)
            if len(meaning)>0:
                meanings.add(meaning)
        meanings = list(meanings)
            
        for acr in acrs:
            if acr in dct:
                dct[acr]+=meanings
            else:
                dct[acr] = meanings
dct

ā
before
before

a
assessment

ap
apical

aa
of each
amino acids
a or ala – alanine
c or cys – cysteine
d or asp – aspartic acid
e or glu – glutamic acid
f
or phe – phenylalanine
h or his – histidine
i or ile – isoleucine
k or lys – lysine
l or leu – leucine
m or met – methionine
n or asn – asparagine
o or pyl – pyrrolysine
p or pro – proline
q or gln – glutamine
r or arg – arginine
s or ser – serine
t or thr – threonine
u or sec – selenocysteine
v or val – valine
w or trp – tryptophan
y or tyr – tyrosine

a1c
glycosylated hemoglobin

aa
arteria see artery

aaa
abdominal aortic aneurysm

aad
antibiotic-associated diarrhea

aai
acute arterial insufficiency

aapmc
antibiotic-associated pseudomembranous colitis

aat
activity as tolerated
atypical antibody test

aal
anterior axillary line

aaox3
awake alert and oriented times 3

a/b
acid-base ratio

ab
abdomen abdominal
abortion

ab
antibody

ab
abortion ab blood type

abc
airway breathing circulation
aspiration biopsy cytology

abcd
airwa

arteriovenous malformation

avn
avascular necrosis
atrioventricular node

avr
aortic valve replacement

avss

afebrile vital signs stable

a&w
a/w
alive and well
associated with
admitted with

ax
axillary

axr
abdominal x-ray

aya
adolescents and young adults

azt
azidothymidine



{'': ['afebrile vital signs stable'],
 'a': ['assessment'],
 'a&o or a/o': ['aware and oriented or alert and oriented'],
 'a&ox3': ['alert and oriented times 3'],
 'a&ox4': ['alert and oriented times 4'],
 'a&p': ['auscultation and percussion', 'anatomy and physiology'],
 'a&w': ['associated with', 'alive and well', 'admitted with'],
 'a+e': ['accident and emergency'],
 'a-t': ['ataxia-telangiectasia'],
 'a/b': ['acid-base ratio'],
 'a/p': ['assessment and plan]]', 'anatomy and physiology'],
 'a/w': ['associated with', 'alive and well', 'admitted with'],
 'a1c': ['glycosylated hemoglobin'],
 'aa': ['p or pro – proline',
  'h or his – histidine',
  'f',
  'amino acids',
  'e or glu – glutamic acid',
  'w or trp – tryptophan',
  'l or leu – leucine',
  'or phe – phenylalanine',
  'm or met – methionine',
  't or thr – threonine',
  'u or sec – selenocysteine',
  'y or tyr – tyrosine',
  'd or asp – aspartic acid',
  'o or pyl – pyrrolysine',
  's or ser – serine',
  'n or asn – asparagin

In [82]:
dct['NS']

KeyError: 'NS'

In [ ]:
print(sorted(dct.items()))

In [ ]:
'n-acetyl cysteine'.upper().lower()

In [105]:
def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size
get_size(dct)

648765